<a href="https://colab.research.google.com/github/ceb263/nhl/blob/main/NHL_Predict_Today.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To do:

LAK not in PlayerGame for 2020-05-10 but are in teamGame

Null values in train_df for game prediction model

SJS missing from PlayerGame 2021-05-12

Fix playerGame game_id

Different loss functions?

Improve use of player data

Goalie sv% or goals saved vs expected?

Use regular-season only for game score by player?

In [1]:
!pip install hockey_scraper

     |████████████████████████████████| 86 kB 2.5 MB/s 


In [2]:
import hockey_scraper
import pandas as pd
import numpy as np
import pickle
from datetime import date, timedelta
from sklearn.metrics import log_loss, roc_auc_score

In [3]:
today = date.today()
yesterday = today - timedelta(days=1)

In [4]:
#pbp = hockey_scraper.scrape_seasons([2021], False, data_format='Pandas')

In [5]:
pbp = hockey_scraper.scrape_date_range("2022-01-18", yesterday.strftime('%Y-%m-%d'), False, data_format='Pandas')
today_df = hockey_scraper.scrape_schedule(today.strftime('%Y-%m-%d'), today.strftime('%Y-%m-%d'))

Scraping Game  2021020340 2022-01-18
Scraping Game  2021020691 2022-01-18
Scraping Game  2021020692 2022-01-18
Scraping Game  2021020695 2022-01-18


Scraping Game  2021020696 2022-01-18
Scraping Game  2021020697 2022-01-18
Scraping Game  2021020698 2022-01-18
Scraping Game  2021020700 2022-01-18

Scraping the schedule between 2022-01-19 and 2022-01-19


In [6]:
pbp = pbp['pbp']
pbp['Season'] = 2021

pbp['Empty_Net'] = (pbp['Away_Goalie'].isnull()) | (pbp['Home_Goalie'].isnull())

# fix team names
pbp['Ev_Team'] = pbp['Ev_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})
pbp['Home_Team'] = pbp['Home_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})
pbp['Away_Team'] = pbp['Away_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})

# remove null location data
pbp = pbp.loc[~pbp['xC'].isnull()]
pbp = pbp.loc[~pbp['yC'].isnull()]

# get previous event time and location
pbp = pbp.sort_values(by=['Game_Id','Period','Seconds_Elapsed'])
pbp['prev_Game_Id'] = pbp['Game_Id'].shift(1)
pbp['prev_Period'] = pbp['Period'].shift(1)
pbp['keepPrev'] = ((pbp['prev_Game_Id']==pbp['Game_Id']) & (pbp['prev_Period']==pbp['Period'])).astype(int)
pbp['prev_Event'] = pbp['Event'].shift(1)
pbp['prev_Seconds_Elapsed'] = pbp['Seconds_Elapsed'].shift(1)
pbp['prev_xC'] = pbp['xC'].shift(1)
pbp['prev_yC'] = pbp['yC'].shift(1)
pbp.at[pbp['keepPrev']==0, ['prev_Event']] = np.NaN
pbp.at[pbp['keepPrev']==0, ['prev_Seconds_Elapsed','prev_xC','prev_yC']] = 0.
#pbp = pbp.loc[~pbp['prev_Event'].isnull()]

# get time elapsed, and distance from previous event
pbp['timeSincePrev'] = pbp['Seconds_Elapsed'] - pbp['prev_Seconds_Elapsed']
pbp['distanceSincePrev'] = np.sqrt(np.square(pbp['xC']-pbp['prev_xC']) + np.square(pbp['yC']-pbp['prev_yC']))
pbp['yDistanceSincePrev'] = np.abs(pbp['yC'] - pbp['prev_yC'])

# remove invalid data
#pbp = pbp.loc[(pbp['timeSincePrev']>0) | (pbp['timeSincePrev'].isnull())]

# filter for only shots and shot attempts
pbp = pbp.loc[pbp['Event'].isin(['BLOCK','MISS','SHOT','GOAL'])]

# filter for only most common game states
pbp = pbp.loc[pbp['Strength'].isin(['5x5','4x5','3x5','5x4','4x4','5x3','4x3','6x5','5x6','3x4','3x3'])]

# field for whether the shooting team has an empty net to shoot at
pbp['ShotIntoEmptyNet'] = (((pbp['Ev_Team']==pbp['Home_Team']) & pbp['Away_Goalie'].isnull()) | ((pbp['Ev_Team']==pbp['Away_Team']) & pbp['Home_Goalie'].isnull())).astype(int)

# get previous shot time and location, and then calculate derived metrics
pbp['prevShot_Game_Id'] = pbp['Game_Id'].shift(1)
pbp['prevShot_Period'] = pbp['Period'].shift(1)
pbp['keepPrevShot'] = ((pbp['prevShot_Game_Id']==pbp['Game_Id']) & (pbp['prevShot_Period']==pbp['Period'])).astype(int)
pbp['prevShot_Seconds_Elapsed'] = pbp['Seconds_Elapsed'].shift(1)
pbp['prevShot_xC'] = pbp['xC'].shift(1)
pbp['prevShot_yC'] = pbp['yC'].shift(1)
pbp['prevShot_Ev_Team'] = pbp['Ev_Team'].shift(1)
pbp['prevShot_sameTeam'] = (pbp['prevShot_Ev_Team']==pbp['Ev_Team']).astype(int)
pbp.at[pbp['keepPrevShot']==0, ['prevShot_Seconds_Elapsed','prevShot_xC','prevShot_yC','prevShot_Ev_Team']] = np.NaN
pbp['timeSincePrevShot'] = pbp['Seconds_Elapsed'] - pbp['prevShot_Seconds_Elapsed']
pbp['distanceSincePrevShot'] = np.sqrt(np.square(pbp['xC']-pbp['prevShot_xC']) + np.square(pbp['yC']-pbp['prevShot_yC']))
pbp['yDistanceSincePrevShot'] = np.abs(pbp['yC'] - pbp['prevShot_yC'])

# adjust shot locations so everything is on the same side of the ice
# TODO this isn't quite right - shots from the D zone (into an empty net, for example), will not be adjusted correctly
pbp['loc_adjust_factor'] = (((pbp['xC']>0).astype(int).astype(float)) - 0.5) * 2
pbp['x_adj'] = pbp['xC']*pbp['loc_adjust_factor']
pbp['y_adj'] = pbp['yC']*pbp['loc_adjust_factor']
pbp['prev_loc_adjust_factor'] = (((pbp['prev_xC']>0).astype(int).astype(float)) - 0.5) * 2
pbp['prev_x_adj'] = pbp['prev_xC']*pbp['prev_loc_adjust_factor']
pbp['prev_y_adj'] = pbp['prev_yC']*pbp['prev_loc_adjust_factor']
pbp['prevShot_loc_adjust_factor'] = (((pbp['prevShot_xC']>0).astype(int).astype(float)) - 0.5) * 2
pbp['prevShot_x_adj'] = pbp['prevShot_xC']*pbp['prevShot_loc_adjust_factor']
pbp['prevShot_y_adj'] = pbp['prevShot_yC']*pbp['prevShot_loc_adjust_factor']

# fill nulls
pbp[['prevShot_Seconds_Elapsed','prevShot_yC','prevShot_y_adj','prevShot_xC']] = \
    pbp[['prevShot_Seconds_Elapsed','prevShot_yC','prevShot_y_adj','prevShot_xC']].fillna(0)
pbp[['prevShot_x_adj','distanceSincePrevShot','yDistanceSincePrevShot']] = pbp[['prevShot_x_adj','distanceSincePrevShot','yDistanceSincePrevShot']].fillna(-1)
pbp[['timeSincePrevShot']] = pbp[['timeSincePrevShot']].fillna(1200)

# fix time since prev shot if prev shot was in another period
pbp.at[pbp['timeSincePrevShot']<0, 'timeSincePrevShot'] = 1200

# adjust score to be score for and against, instead of home and away
pbp['homeTeamShot'] = (pbp['Home_Team']==pbp['Ev_Team']).astype(int)
pbp['scoreFor'] = (pbp['Home_Score']*pbp['homeTeamShot']) + (pbp['Away_Score']*(1-pbp['homeTeamShot']))
pbp['scoreAgainst'] = (pbp['Away_Score']*pbp['homeTeamShot']) + (pbp['Home_Score']*(1-pbp['homeTeamShot']))
pbp['scoreDiff'] = pbp['scoreFor'] - pbp['scoreAgainst']

# add target variable
pbp['goal'] = (pbp['Event']=='GOAL').astype(int)

In [7]:
mean_encodings = pickle.load(open('xG_mean_encodings.pkl', 'rb'))

In [8]:
pbp['Strength_meanEnc'] = pbp['Strength'].map(mean_encodings['strength'])
pbp['Ev_Zone_meanEnc'] = pbp['Ev_Zone'].map(mean_encodings['zone'])
pbp['Type_meanEnc'] = pbp['Type'].map(mean_encodings['type'])
pbp['prev_Event_meanEnc'] = pbp['prev_Event'].map(mean_encodings['prevEvent'])

In [9]:
xg_scaler = pickle.load(open('xG_scaler.pkl', 'rb'))

Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [10]:
xg_model = pickle.load(open('xG_gb.pkl', 'rb'))

Trying to unpickle estimator DummyClassifier from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator DecisionTreeRegressor from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator GradientBoostingClassifier from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [11]:
features = ['Period','Seconds_Elapsed','scoreDiff','yC','prev_Seconds_Elapsed','timeSincePrev','distanceSincePrev',
           'prevShot_Seconds_Elapsed','prevShot_sameTeam','timeSincePrevShot','ShotIntoEmptyNet',
           'x_adj','y_adj','Strength_meanEnc','Ev_Zone_meanEnc','Type_meanEnc']
xg_x = xg_scaler.transform(pbp[features].values)
pbp['xG'] = xg_model.predict_proba(xg_x)[:,1]

# Team Aggregation

In [12]:
# calculate some stats in the disaggregated data
pbp['Goals'] = ((pbp['Event'] == 'GOAL') & (pbp['Seconds_Elapsed']!=0)).astype(np.int16)
pbp['Shootout_Goals'] = ((pbp['Event'] == 'GOAL') & (pbp['Seconds_Elapsed']==0)).astype(np.int16)
pbp['Shots'] = ((pbp['Event'].isin(['SHOT','GOAL'])) & (pbp['Seconds_Elapsed']!=0)).astype(np.int16)
pbp['ShotAttempts'] = ((pbp['Event'].isin(['SHOT','MISS','GOAL','BLOCK'])) & (pbp['Seconds_Elapsed']!=0)).astype(np.int16)
pbp['Goals_5v5'] = ((pbp['Event'] == 'GOAL') & (pbp['Strength']=='5x5')).astype(np.int16)
pbp['Shots_5v5'] = ((pbp['Event'].isin(['SHOT','GOAL'])) & (pbp['Strength']=='5x5')).astype(np.int16)
pbp['ShotAttempts_5v5'] = ((pbp['Event'].isin(['SHOT','MISS','GOAL','BLOCK'])) & (pbp['Strength']=='5x5')).astype(np.int16)
pbp['xG_5v5'] = np.nan
pbp.loc[pbp['Strength']=='5x5', 'xG_5v5'] = pbp.loc[pbp['Strength']=='5x5']['xG']
pbp['Penalties'] = (pbp['Event'] == 'PENL').astype(np.int16)
pbp['Home'] = (pbp['Ev_Team']==pbp['Home_Team']).astype(np.int16)

# create aggregate
teamGame = pbp[[
        'Game_Id','Date','Ev_Team','Season','Goals','Shootout_Goals','Shots','ShotAttempts','xG','Goals_5v5','Shots_5v5','ShotAttempts_5v5','xG_5v5','Penalties','Home'
    ]].groupby([
        'Game_Id','Date','Ev_Team','Season'
    ]).agg({
        'Goals' : 'sum',
        'Shootout_Goals' : 'sum',
        'Shots' : 'sum',
        'ShotAttempts' : 'sum',
        'xG' : 'sum',
        'Goals_5v5' : 'sum',
        'Shots_5v5' : 'sum',
        'ShotAttempts_5v5' : 'sum',
        'xG_5v5' : 'sum',
        'Penalties' : 'sum',
        'Home' : 'max'
    }).reset_index()
teamGame = teamGame.rename(columns={'Ev_Team':'Team'})
teamGame = teamGame.fillna(0)

# add win column
df_temp = teamGame.groupby(['Game_Id','Date']).agg({'Goals':'max', 'Shootout_Goals':'max'}).reset_index()
df_temp = df_temp.rename(columns={'Goals':'winGoals', 'Shootout_Goals':'winShootoutGoals'})
teamGame = teamGame.merge(df_temp, on=['Game_Id','Date'])
del df_temp
teamGame['Win'] = ((teamGame['Goals']==teamGame['winGoals']) & (teamGame['Shootout_Goals']==teamGame['winShootoutGoals'])).astype(np.int16)
teamGame = teamGame.drop(['Shootout_Goals','winGoals','winShootoutGoals'],1)

# add field for whether this is a playoff game or not
teamGame['DateInt'] = teamGame['Date'].str.replace('-','').astype(np.int64)
teamGame['teamGameRank'] = teamGame.groupby(['Team'])['DateInt'].rank("dense")
teamGame['Playoffs'] = (teamGame['teamGameRank']>82).astype(np.int8)
teamGame.loc[(teamGame['Season']==2012)&(teamGame['teamGameRank']>48),'Playoffs'] = 1 #fix for the lockout-shortened season
teamGame.loc[(teamGame['Season']==2019)&(teamGame['Date']>'2020-03-12'),'Playoffs'] = 1 #fix for the first covid-shortened season
teamGame.loc[(teamGame['Season']==2020)&(teamGame['teamGameRank']>56),'Playoffs'] = 1 #fix for the second covid-shortened season

teamGame = teamGame.loc[teamGame['Team'].str.len()==3]

teamGame = teamGame.sort_values(by=['DateInt','Game_Id'])
teamGame['Game_Id'] = (teamGame['Season'].astype(str) + teamGame['Game_Id'].astype(str)).astype(np.int64)

In [13]:
teamGameOld = pd.read_csv('teamGame.gz')
#teamGameOld = teamGameOld.loc[teamGameOld['Date']<"2022-01-13"]

In [14]:
teamGameOld['Date'].max()

'2022-01-17'

In [15]:
#teamGame = teamGameOld.copy()

In [16]:
today_home = today_df[['game_id','date','home_team']].copy()
today_home.columns = ['Game_Id','Date','Team']
today_home['Home'] = 1
today_away = today_df[['game_id','date','away_team']].copy()
today_away.columns = ['Game_Id','Date','Team']
today_away['Home'] = 0
today_df = pd.concat([today_home, today_away], ignore_index=True)
today_df['Team'] = today_df['Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})
today_df['Season'] = today_df['Game_Id'].astype(str).str[:4].astype(int)

In [17]:
teamGame = pd.concat([teamGame, today_df], ignore_index=True)
#teamGame = today_df.copy()

teamGame['DateInt'] = teamGame['Date'].str.replace('-','').astype(np.int64)
teamGame['teamGameRank'] = teamGame.groupby(['Team'])['DateInt'].rank("dense")

In [18]:
latestGames = teamGameOld.loc[teamGameOld['Game_Id'].isin(teamGameOld.groupby(['Team']).agg({'Game_Id':'max'})['Game_Id'])]

In [19]:
latestRanks = latestGames.groupby('Team').agg({'teamGameRankOverall':'max'})
latestRanks = latestRanks.to_dict()['teamGameRankOverall']

In [20]:
teamGame['latestRank'] = teamGame['Team'].map(latestRanks)
teamGame['teamGameRankOverall'] = teamGame['teamGameRank'] + teamGame['latestRank'].fillna(0)
teamGame = teamGame.drop('latestRank',1)
teamGame['Elo'] = np.nan
teamGame['keep'] = True
latestGames['keep'] = False

teamGame = pd.concat([latestGames,teamGame], ignore_index=True)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
for s in teamGame.loc[teamGame['keep']]['Season'].unique():
    teamGameRankStarts = teamGame.loc[(~teamGame['keep']) & (teamGame['Season']==s)].groupby('Team').agg({'teamGameRank':'max'})
    teamGameRankStarts = teamGameRankStarts.to_dict()['teamGameRank']
    teamGame.loc[(teamGame['keep']) & (teamGame['Season']==s), 'teamGameRank'] = teamGame.loc[(teamGame['keep']) & (teamGame['Season']==s), 'teamGameRank'] + \
        teamGame['Team'].map(teamGameRankStarts)

In [22]:
# add elo ratings
# https://fivethirtyeight.com/methodology/how-our-nhl-predictions-work/
teamGame = teamGame.sort_values(by=['DateInt','Game_Id'])
teamGame.loc[(teamGame['teamGameRankOverall']==1)&(teamGame['Elo'].isnull()),'Elo'] = 1500
for gid in teamGame['Game_Id'].unique():
    game_df = teamGame.loc[teamGame['Game_Id']==gid].copy()
    elo_1 = game_df['Elo'].iloc[0]
    elo_2 = game_df['Elo'].iloc[1]
    if game_df['teamGameRank'].iloc[0]==1:
        elo_1 = (elo_1*0.7) + (1505*0.3)
        teamGame.loc[(teamGame['Team']==game_df['Team'].iloc[0]) & (teamGame['teamGameRankOverall']==game_df['teamGameRankOverall'].iloc[0]), 'Elo'] = elo_1
    if game_df['teamGameRank'].iloc[1]==1:
        elo_2 = (elo_2*0.7) + (1505*0.3)
        teamGame.loc[(teamGame['Team']==game_df['Team'].iloc[1]) & (teamGame['teamGameRankOverall']==game_df['teamGameRankOverall'].iloc[1]), 'Elo'] = elo_2
    p_1 = 1/(10**((elo_2-elo_1)/400)+1)
    p_2 = 1/(10**((elo_1-elo_2)/400)+1)
    win_1 = game_df['Win'].iloc[0]
    win_2 = game_df['Win'].iloc[1]
    victoryMarginMultiplier_1 = 0.6686 * np.log(np.max([np.abs(game_df['Goals'].iloc[0]-game_df['Goals'].iloc[1]),1])) + 0.8048
    victoryMarginMultiplier_2 = 0.6686 * np.log(np.max([np.abs(game_df['Goals'].iloc[1]-game_df['Goals'].iloc[0]),1])) + 0.8048
    if elo_1>elo_2 and win_1==1:
        autoAdjust_1 = 2.05/((elo_1-elo_2) * 0.001 + 2.05)
        autoAdjust_2 = 1
    elif elo_2>elo_1 and win_2==1:
        autoAdjust_2 = 2.05/((elo_1-elo_2) * 0.001 + 2.05)
        autoAdjust_1 = 1
    else:
        autoAdjust_1 = 1
        autoAdjust_2 = 1
    favMultiplier_1 = win_1 - p_1
    favMultiplier_2 = win_2 - p_2
    new_elo_1 = elo_1 + (6*victoryMarginMultiplier_1*autoAdjust_1*favMultiplier_1)
    new_elo_2 = elo_2 + (6*victoryMarginMultiplier_2*autoAdjust_2*favMultiplier_2)
    teamGame.loc[(teamGame['Team']==game_df['Team'].iloc[0]) & (teamGame['teamGameRankOverall']==game_df['teamGameRankOverall'].iloc[0]+1), 'Elo'] = new_elo_1
    teamGame.loc[(teamGame['Team']==game_df['Team'].iloc[1]) & (teamGame['teamGameRankOverall']==game_df['teamGameRankOverall'].iloc[1]+1), 'Elo'] = new_elo_2

In [23]:
teamGame = teamGame.loc[teamGame['keep']]
teamGame = teamGame.drop('keep',1)
teamGame = pd.concat([teamGameOld, teamGame], ignore_index=True)
teamGame['teamGameRank'] = teamGame.groupby(['Team','Season'])['DateInt'].rank("dense")

In [24]:
teamGame.loc[teamGame['Date']<today.strftime('%Y-%m-%d')].to_csv('teamGame.gz', compression='gzip', index=False)

# Player Aggregation

In [25]:
# aggregate by player, team, and game
def _aggregate_playergame(plays):
    # fix ARZ/PHX and other team names
    plays['Ev_Team'] = plays['Ev_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})
    plays['Home_Team'] = plays['Home_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})
    plays['Away_Team'] = plays['Away_Team'].replace({'PHX':'ARI', 'S.J':'SJS', 'L.A':'LAK', 'T.B':'TBL', 'N.J':'NJD'})

    # calculate some stats in the disaggregated data, to sum later
    plays['Goals'] = ((plays['Event'] == 'GOAL') & (plays['Strength']!='0x0')).astype(np.int16)
    plays['Shootout_Goals'] = ((plays['Event'] == 'GOAL') & (plays['Strength']=='0x0')).astype(np.int16)
    plays['Shots'] = ((plays['Event'].isin(['SHOT','GOAL'])) & (plays['Strength']!='0x0')).astype(np.int16)
    plays['ShotAttempts'] = ((plays['Event'].isin(['SHOT','MISS','GOAL','BLOCK'])) & (plays['Strength']!='0x0')).astype(np.int16)
    plays['Goals_evenstrength'] = ((plays['Event'] == 'GOAL') & (plays['Strength'].isin(['5x5','4x4','3x3']))).astype(np.int16)
    plays['Shots_evenstrength'] = ((plays['Event'].isin(['SHOT','GOAL'])) & (plays['Strength'].isin(['5x5','4x4','3x3']))).astype(np.int16)
    plays['ShotAttempts_evenstrength'] = ((plays['Event'].isin(['SHOT','MISS','GOAL','BLOCK'])) & (plays['Strength'].isin(['5x5','4x4','3x3']))).astype(np.int16)
    plays['xG_evenstrength'] = np.nan
    plays.loc[plays['Strength'].isin(['5x5','4x4','3x3']), 'xG_evenstrength'] = plays.loc[plays['Strength'].isin(['5x5','4x4','3x3'])]['xG']
    plays['Penalties'] = (plays['Event'] == 'PENL').astype(np.int16)
    plays['Home_Starts_Off'] = ((plays['Event'] == 'FAC') & (plays['Home_Zone']=='Off') & (plays['Strength'].isin(['5x5','4x4']))).astype(np.int16)
    plays['Home_Starts_Def'] = ((plays['Event'] == 'FAC') & (plays['Home_Zone']=='Def') & (plays['Strength'].isin(['5x5','4x4']))).astype(np.int16)

    # create field for player that should get credited with the event
    plays['Player'] = np.nan
    plays['PlayerID'] = np.nan
    plays.loc[plays['Event']=='BLOCK', 'Player'] = plays.loc[plays['Event']=='BLOCK']['p2_name']
    plays.loc[plays['Event']=='BLOCK', 'PlayerID'] = plays.loc[plays['Event']=='BLOCK']['p2_ID']
    plays.loc[plays['Event'].isin(['SHOT','MISS','GOAL']), 'Player'] = plays.loc[plays['Event'].isin(['SHOT','MISS','GOAL'])]['p1_name']
    plays.loc[plays['Event'].isin(['SHOT','MISS','GOAL']), 'PlayerID'] = plays.loc[plays['Event'].isin(['SHOT','MISS','GOAL'])]['p1_ID']
    plays.loc[plays['Event']=='PENL', 'Player'] = plays.loc[plays['Event']=='PENL']['p1_name']
    plays.loc[plays['Event']=='PENL', 'PlayerID'] = plays.loc[plays['Event']=='PENL']['p1_ID']

    # create first aggregate
    playerGame = plays.groupby([
            'Game_Id','Date','Player','PlayerID','Season'
        ]).agg({
            'Goals' : sum,
            'Shootout_Goals' : sum,
            'xG' : sum,
            'Shots' : sum,
            'ShotAttempts' : sum,
            'Goals_evenstrength' : sum,
            'Shots_evenstrength' : sum,
            'ShotAttempts_evenstrength' : sum,
            'xG_evenstrength' : sum,
            'Penalties' : sum
    }).reset_index()

    # second aggregate for primary assists and penalties drawn
    plays = plays.rename(columns={'Goals' : 'PrimaryAssists', 'Goals_evenstrength' : 'PrimaryAssists_evenstrength', 'Penalties' : 'PenaltiesDrawn'})
    plays.loc[:,'Player'] = plays['p2_name']
    plays.loc[:,'PlayerID'] = plays['p2_ID']
    playerGame_2 = plays.groupby([
            'Game_Id','Date','Player','PlayerID','Season'
        ]).agg({
            'PrimaryAssists' : sum,
            'PrimaryAssists_evenstrength' : sum,
            'PenaltiesDrawn' : sum
    }).reset_index()
    playerGame = playerGame.merge(playerGame_2, how='left', on=['Game_Id','Date','Player','PlayerID','Season'])
    del playerGame_2

    # third aggregate for secondary assists
    plays = plays.rename(columns={'PrimaryAssists' : 'SecondaryAssists', 'PrimaryAssists_evenstrength' : 'SecondaryAssists_evenstrength'})
    plays.loc[:,'Player'] = plays['p3_name']
    plays.loc[:,'PlayerID'] = plays['p3_ID']
    playerGame_3 = plays.groupby([
            'Game_Id','Date','Player','PlayerID','Season'
        ]).agg({
            'SecondaryAssists' : sum,
            'SecondaryAssists_evenstrength' : sum
    }).reset_index()
    playerGame = playerGame.merge(playerGame_3, how='left', on=['Game_Id','Date','Player','PlayerID','Season'])
    del playerGame_3

    # prep data for on-ice aggregates
    shots = plays.loc[plays['Event'].isin(['SHOT','MISS','GOAL','BLOCK','FAC'])]
    shots = shots.rename(columns={'SecondaryAssists' : 'Goals', 'SecondaryAssists_evenstrength' : 'Goals_evenstrength', 'PenaltiesDrawn' : 'Penalties'})
    del plays # to save some memory
    shots['HomeTeamEvent'] = (shots['Ev_Team']==shots['Home_Team']).astype(int)
    shots['AwayTeamEvent'] = (shots['Ev_Team']==shots['Away_Team']).astype(int)

    # on-ice stats home players
    for i in range(1,7):
        shots['Player'] = shots['homePlayer{}'.format(str(i))].copy()
        shots['PlayerID'] = shots['homePlayer{}_id'.format(str(i))].copy()

        shots['Goals_onice_home{}'.format(str(i))] = shots['Goals'] * shots['HomeTeamEvent']
        shots['Shots_onice_home{}'.format(str(i))] = shots['Shots'] * shots['HomeTeamEvent']
        shots['ShotAttempts_onice_home{}'.format(str(i))] = shots['ShotAttempts'] * shots['HomeTeamEvent']
        shots['xG_onice_home{}'.format(str(i))] = shots['xG'] * shots['HomeTeamEvent']
        shots['Goals_evenstrength_onice_home{}'.format(str(i))] = shots['Goals_evenstrength'] * shots['HomeTeamEvent']
        shots['Shots_evenstrength_onice_home{}'.format(str(i))] = shots['Shots_evenstrength'] * shots['HomeTeamEvent']
        shots['ShotAttempts_evenstrength_onice_home{}'.format(str(i))] = shots['ShotAttempts_evenstrength'] * shots['HomeTeamEvent']
        shots['xG_evenstrength_onice_home{}'.format(str(i))] = shots['xG_evenstrength'] * shots['HomeTeamEvent']

        shots['GoalsAgainst_onice_home{}'.format(str(i))] = shots['Goals'] * shots['AwayTeamEvent']
        shots['ShotsAgainst_onice_home{}'.format(str(i))] = shots['Shots'] * shots['AwayTeamEvent']
        shots['ShotAttemptsAgainst_onice_home{}'.format(str(i))] = shots['ShotAttempts'] * shots['AwayTeamEvent']
        shots['xGAgainst_onice_home{}'.format(str(i))] = shots['xG'] * shots['AwayTeamEvent']
        shots['GoalsAgainst_evenstrength_onice_home{}'.format(str(i))] = shots['Goals_evenstrength'] * shots['AwayTeamEvent']
        shots['ShotsAgainst_evenstrength_onice_home{}'.format(str(i))] = shots['Shots_evenstrength'] * shots['AwayTeamEvent']
        shots['ShotAttemptsAgainst_evenstrength_onice_home{}'.format(str(i))] = shots['ShotAttempts_evenstrength'] * shots['AwayTeamEvent']
        shots['xGAgainst_evenstrength_onice_home{}'.format(str(i))] = shots['xG_evenstrength'] * shots['AwayTeamEvent']

        shots['team_home{}'.format(str(i))] = shots['Home_Team'].copy()
        shots['Starts_Off_home{}'.format(str(i))] = shots['Home_Starts_Off'].copy()
        shots['Starts_Def_home{}'.format(str(i))] = shots['Home_Starts_Def'].copy()

        # add column for tracking player positions
        shots['position_home{}'.format(str(i))] = 1

        playerGame_home = shots.groupby([
                'Game_Id','Date','Player','PlayerID','Season'
            ]).agg({
                'Goals_onice_home{}'.format(str(i)) : sum,
                'Shots_onice_home{}'.format(str(i)) : sum,
                'ShotAttempts_onice_home{}'.format(str(i)) : sum,
                'xG_onice_home{}'.format(str(i)) : sum,
                'Goals_evenstrength_onice_home{}'.format(str(i)) : sum,
                'Shots_evenstrength_onice_home{}'.format(str(i)) : sum,
                'ShotAttempts_evenstrength_onice_home{}'.format(str(i)) : sum,
                'xG_evenstrength_onice_home{}'.format(str(i)) : sum,
                'GoalsAgainst_onice_home{}'.format(str(i)) : sum,
                'ShotsAgainst_onice_home{}'.format(str(i)) : sum,
                'ShotAttemptsAgainst_onice_home{}'.format(str(i)) : sum,
                'xGAgainst_onice_home{}'.format(str(i)) : sum,
                'GoalsAgainst_evenstrength_onice_home{}'.format(str(i)) : sum,
                'ShotsAgainst_evenstrength_onice_home{}'.format(str(i)) : sum,
                'ShotAttemptsAgainst_evenstrength_onice_home{}'.format(str(i)) : sum,
                'xGAgainst_evenstrength_onice_home{}'.format(str(i)) : sum,
                'position_home{}'.format(str(i)) : sum,
                'team_home{}'.format(str(i)) : lambda x: x.value_counts().index[0],
                'Starts_Off_home{}'.format(str(i)) : sum,
                'Starts_Def_home{}'.format(str(i)) : sum
        }).reset_index().fillna(0)
        playerGame_home['team_home{}'.format(str(i))] = playerGame_home['team_home{}'.format(str(i))].replace(0, '')

        playerGame = playerGame.merge(playerGame_home, how='left', on=['Game_Id','Date','Player','PlayerID','Season'])
        del playerGame_home

    # on-ice stats away players
    for i in range(1,7):
        shots['Player'] = shots['awayPlayer{}'.format(str(i))].copy()
        shots['PlayerID'] = shots['awayPlayer{}_id'.format(str(i))].copy()

        shots['Goals_onice_away{}'.format(str(i))] = shots['Goals'] * shots['AwayTeamEvent']
        shots['Shots_onice_away{}'.format(str(i))] = shots['Shots'] * shots['AwayTeamEvent']
        shots['ShotAttempts_onice_away{}'.format(str(i))] = shots['ShotAttempts'] * shots['AwayTeamEvent']
        shots['xG_onice_away{}'.format(str(i))] = shots['xG'] * shots['AwayTeamEvent']
        shots['Goals_evenstrength_onice_away{}'.format(str(i))] = shots['Goals_evenstrength'] * shots['AwayTeamEvent']
        shots['Shots_evenstrength_onice_away{}'.format(str(i))] = shots['Shots_evenstrength'] * shots['AwayTeamEvent']
        shots['ShotAttempts_evenstrength_onice_away{}'.format(str(i))] = shots['ShotAttempts_evenstrength'] * shots['AwayTeamEvent']
        shots['xG_evenstrength_onice_away{}'.format(str(i))] = shots['xG_evenstrength'] * shots['AwayTeamEvent']

        shots['GoalsAgainst_onice_away{}'.format(str(i))] = shots['Goals'] * shots['HomeTeamEvent']
        shots['ShotsAgainst_onice_away{}'.format(str(i))] = shots['Shots'] * shots['HomeTeamEvent']
        shots['ShotAttemptsAgainst_onice_away{}'.format(str(i))] = shots['ShotAttempts'] * shots['HomeTeamEvent']
        shots['xGAgainst_onice_away{}'.format(str(i))] = shots['xG'] * shots['HomeTeamEvent']
        shots['GoalsAgainst_evenstrength_onice_away{}'.format(str(i))] = shots['Goals_evenstrength'] * shots['HomeTeamEvent']
        shots['ShotsAgainst_evenstrength_onice_away{}'.format(str(i))] = shots['Shots_evenstrength'] * shots['HomeTeamEvent']
        shots['ShotAttemptsAgainst_evenstrength_onice_away{}'.format(str(i))] = shots['ShotAttempts_evenstrength'] * shots['HomeTeamEvent']
        shots['xGAgainst_evenstrength_onice_away{}'.format(str(i))] = shots['xG_evenstrength'] * shots['HomeTeamEvent']

        shots['team_away{}'.format(str(i))] = shots['Away_Team'].copy()
        shots['Starts_Off_away{}'.format(str(i))] = shots['Home_Starts_Def'].copy()
        shots['Starts_Def_away{}'.format(str(i))] = shots['Home_Starts_Off'].copy()

        # add column for tracking player positions
        shots['position_away{}'.format(str(i))] = 1

        playerGame_away = shots.groupby([
                'Game_Id','Date','Player','PlayerID','Season'
            ]).agg({
                'Goals_onice_away{}'.format(str(i)) : sum,
                'Shots_onice_away{}'.format(str(i)) : sum,
                'ShotAttempts_onice_away{}'.format(str(i)) : sum,
                'xG_onice_away{}'.format(str(i)) : sum,
                'Goals_evenstrength_onice_away{}'.format(str(i)) : sum,
                'Shots_evenstrength_onice_away{}'.format(str(i)) : sum,
                'ShotAttempts_evenstrength_onice_away{}'.format(str(i)) : sum,
                'xG_evenstrength_onice_away{}'.format(str(i)) : sum,
                'GoalsAgainst_onice_away{}'.format(str(i)) : sum,
                'ShotsAgainst_onice_away{}'.format(str(i)) : sum,
                'ShotAttemptsAgainst_onice_away{}'.format(str(i)) : sum,
                'xGAgainst_onice_away{}'.format(str(i)) : sum,
                'GoalsAgainst_evenstrength_onice_away{}'.format(str(i)) : sum,
                'ShotsAgainst_evenstrength_onice_away{}'.format(str(i)) : sum,
                'ShotAttemptsAgainst_evenstrength_onice_away{}'.format(str(i)) : sum,
                'xGAgainst_evenstrength_onice_away{}'.format(str(i)) : sum,
                'position_away{}'.format(str(i)) : sum,
                'team_away{}'.format(str(i)) : lambda x: x.value_counts().index[0],
                'Starts_Off_away{}'.format(str(i)) : sum,
                'Starts_Def_away{}'.format(str(i)) : sum
        }).reset_index()
        playerGame_away['team_away{}'.format(str(i))] = playerGame_away['team_away{}'.format(str(i))].replace(0, '')

        playerGame = playerGame.merge(playerGame_away, how='left', on=['Game_Id','Date','Player','PlayerID','Season'])
        del playerGame_away

        # add up home and away player position numbers
        playerGame['position_{}'.format(str(i))] = playerGame['position_home{}'.format(str(i))].fillna(0) + playerGame['position_away{}'.format(str(i))].fillna(0)
        playerGame = playerGame.drop(['position_home{}'.format(str(i)),'position_away{}'.format(str(i))], 1)

    # fill nulls, and add up on-ice columns so there is only one per stat
    playerGame = playerGame.fillna(0)
    for col in ['Goals_onice','Shots_onice','ShotAttempts_onice','xG_onice','Goals_evenstrength_onice','Shots_evenstrength_onice',
                'ShotAttempts_evenstrength_onice','xG_evenstrength_onice','GoalsAgainst_onice','ShotsAgainst_onice','ShotAttemptsAgainst_onice',
                'xGAgainst_onice','GoalsAgainst_evenstrength_onice','ShotsAgainst_evenstrength_onice','ShotAttemptsAgainst_evenstrength_onice',
                'xGAgainst_evenstrength_onice','Starts_Off','Starts_Def']:
        playerGame[col] = playerGame['{}_home1'.format(col)] + playerGame['{}_home2'.format(col)] + playerGame['{}_home3'.format(col)] + playerGame['{}_home4'.format(col)] \
            + playerGame['{}_home5'.format(col)] + playerGame['{}_home6'.format(col)] + playerGame['{}_away1'.format(col)] + playerGame['{}_away2'.format(col)] \
            + playerGame['{}_away3'.format(col)] + playerGame['{}_away4'.format(col)] + playerGame['{}_away5'.format(col)] + playerGame['{}_away6'.format(col)]
        playerGame = playerGame.drop(['{}_home1'.format(col),'{}_home2'.format(col),'{}_home3'.format(col),'{}_home4'.format(col),'{}_home5'.format(col),
                                      '{}_home6'.format(col),'{}_away1'.format(col),'{}_away2'.format(col),'{}_away3'.format(col),'{}_away4'.format(col),
                                      '{}_away5'.format(col),'{}_away6'.format(col)], 1)
        
    # combine the team columns into one
    playerGame['Team'] = playerGame[['team_away1','team_away2','team_away3','team_away4','team_away5','team_away6',
                                     'team_home1','team_home2','team_home3','team_home4','team_home5','team_home6']].replace(0,'').max(1)
    playerGame = playerGame.drop(['team_away1','team_away2','team_away3','team_away4','team_away5','team_away6',
                                  'team_home1','team_home2','team_home3','team_home4','team_home5','team_home6'], 1)

    return playerGame

In [26]:
# get the position (Forward, Defence, or Goalie) for each player and add to the aggregate
def _get_position(playerGame):
    players = playerGame[['Player','PlayerID','position_1','position_2','position_3','position_4','position_5','position_6']].groupby([
        'Player','PlayerID'
    ]).sum()
    players['maxcol'] = players.idxmax(1)
    players = players.reset_index()
    players['Position'] = np.nan
    players.loc[players['maxcol'].isin(['position_1','position_2','position_3']),'Position'] = 'F'
    players.loc[players['maxcol'].isin(['position_4','position_5']),'Position'] = 'D'
    players.loc[players['maxcol']=='position_6','Position'] = 'G'

    playerGame = playerGame.drop(['position_1','position_2','position_3','position_4','position_5','position_6'], 1)
    playerGame = playerGame.merge(players[['Player','PlayerID','Position']], on=['Player','PlayerID'], how='left')
    return playerGame

In [27]:
playerGame = _aggregate_playergame(pbp)

In [28]:
playerGame = _get_position(playerGame)

In [29]:
# downcast some fields
playerGame['Game_Id'] = playerGame['Game_Id'].astype(np.int32)
playerGame['PlayerID'] = playerGame['PlayerID'].astype(np.int32)
playerGame['Season'] = playerGame['Season'].astype(np.int16)
playerGame['PrimaryAssists'] = playerGame['PrimaryAssists'].astype(np.int16)
playerGame['PrimaryAssists_evenstrength'] = playerGame['PrimaryAssists_evenstrength'].astype(np.int16)
playerGame['PenaltiesDrawn'] = playerGame['PenaltiesDrawn'].astype(np.int32)
playerGame['SecondaryAssists'] = playerGame['SecondaryAssists'].astype(np.int16)
playerGame['SecondaryAssists_evenstrength'] = playerGame['SecondaryAssists_evenstrength'].astype(np.int16)
playerGame['Goals_onice'] = playerGame['Goals_onice'].astype(np.int16)
playerGame['Shots_onice'] = playerGame['Shots_onice'].astype(np.int32)
playerGame['ShotAttempts_onice'] = playerGame['ShotAttempts_onice'].astype(np.int32)
playerGame['Goals_evenstrength_onice'] = playerGame['Goals_evenstrength_onice'].astype(np.int16)
playerGame['Shots_evenstrength_onice'] = playerGame['Shots_evenstrength_onice'].astype(np.int32)
playerGame['ShotAttempts_evenstrength_onice'] = playerGame['ShotAttempts_evenstrength_onice'].astype(np.int32)
playerGame['GoalsAgainst_onice'] = playerGame['GoalsAgainst_onice'].astype(np.int16)
playerGame['ShotsAgainst_onice'] = playerGame['ShotsAgainst_onice'].astype(np.int32)
playerGame['ShotAttemptsAgainst_onice'] = playerGame['ShotAttemptsAgainst_onice'].astype(np.int32)
playerGame['GoalsAgainst_evenstrength_onice'] = playerGame['GoalsAgainst_evenstrength_onice'].astype(np.int16)
playerGame['ShotsAgainst_evenstrength_onice'] = playerGame['ShotsAgainst_evenstrength_onice'].astype(np.int32)
playerGame['ShotAttemptsAgainst_evenstrength_onice'] = playerGame['ShotAttemptsAgainst_evenstrength_onice'].astype(np.int32)
playerGame['Starts_Off'] = playerGame['Starts_Off'].astype(np.int32)
playerGame['Starts_Def'] = playerGame['Starts_Def'].astype(np.int32)

In [30]:
# add an ID field
playerGame['PlayerGameID'] = playerGame['Date'] + '_' + playerGame['PlayerID'].astype(str)

# add field for whether this is a playoff game or not
playerGame['DateInt'] = playerGame['Date'].str.replace('-','').astype(np.int64)
playerGame['teamGameRank'] = playerGame.groupby(['Team','Season'])['DateInt'].rank("dense")
playerGame['Playoffs'] = (playerGame['teamGameRank']>82).astype(np.int8)
playerGame.loc[(playerGame['Season']==2012)&(playerGame['teamGameRank']>48),'Playoffs'] = 1 #fix for the lockout-shortened season
playerGame = playerGame.drop('teamGameRank',1)

In [31]:
# fix a bug
playerGame = playerGame.loc[playerGame['PlayerID']>=1]

In [32]:
# prep training and testing data

# create target variable, based on game score
# 0.75*goals + 0.7*primary assists + 0.55*secondary assists + 0.075*shots + 
# 0.05*blocks + 0.15*penalty diff + 0.01*faceoff diff + 0.05*5v5 shots diff + 0.15*5v5 goal diff

playerGame['GameScore'] = (0.75*playerGame['Goals']) + (0.7*playerGame['PrimaryAssists']) + (0.55*playerGame['SecondaryAssists']) \
    + (0.075*playerGame['Shots']) + (0.15*(playerGame['PenaltiesDrawn']-playerGame['Penalties'])) \
    + (0.05*(playerGame['Shots_evenstrength_onice']-playerGame['ShotsAgainst_evenstrength_onice'])) \
    + (0.15*(playerGame['Goals_evenstrength_onice']-playerGame['GoalsAgainst_evenstrength_onice'])) \
    + (0.15*(playerGame['xG_evenstrength_onice']-playerGame['xGAgainst_evenstrength_onice']))
#playerGame['GameScore'] = playerGame['Goals_evenstrength_onice']-playerGame['GoalsAgainst_evenstrength_onice']

In [33]:
playerGameOld = pd.read_csv('playerGame.gz')
#playerGameOld = playerGameOld.loc[playerGameOld['Date']<"2022-01-13"]

In [34]:
#playerGame = playerGameOld.copy()

In [35]:
playerGame['Game_Id'] = (playerGame['Season'].astype(str) + playerGame['Game_Id'].astype(str)).astype(np.int64)

In [36]:
playerGame = pd.concat([playerGameOld, playerGame], ignore_index=True)
playerGame['teamGameRank'] = playerGame.groupby(['Team','Season'])['DateInt'].rank("dense")
playerGame.to_csv('playerGame.gz', compression='gzip', index=False)

# Predict New Games

In [37]:
playerGame = playerGame.sort_values(by=['Date','Team'])
playerGame['GameScore_last8'] = playerGame.groupby(['Player','PlayerID','Position'])[['GameScore']].transform(lambda x: x.rolling(window=8).mean()).fillna(0.)
playerGame['GameScore_last16'] = playerGame.groupby(['Player','PlayerID','Position'])[['GameScore']].transform(lambda x: x.rolling(window=16).mean()).fillna(0.)
playerGame['GameScore_last32'] = playerGame.groupby(['Player','PlayerID','Position'])[['GameScore']].transform(lambda x: x.rolling(window=32).mean()).fillna(0.)
playerGame['GameScore_last64'] = playerGame.groupby(['Player','PlayerID','Position'])[['GameScore']].transform(lambda x: x.rolling(window=64).mean()).fillna(0.)

In [38]:
playerTeamGame = playerGame.groupby(['Date','Team']).agg({
    'GameScore' : ['mean','median','max','std'],
    'GameScore_last8' : ['mean','median','max','std'],
    'GameScore_last16' : ['mean','median','max','std'],
    'GameScore_last32' : ['mean','median','max','std'],
    'GameScore_last64' : ['mean','median','max','std']
}).reset_index()
playerTeamGame.columns = ['Date','Team','GameScore_mean','GameScore_median','GameScore_max','GameScore_std',
                          'GameScore_last8_mean','GameScore_last8_median','GameScore_last8_max','GameScore_last8_std',
                          'GameScore_last16_mean','GameScore_last16_median','GameScore_last16_max','GameScore_last16_std',
                          'GameScore_last32_mean','GameScore_last32_median','GameScore_last32_max','GameScore_last32_std',
                          'GameScore_last64_mean','GameScore_last64_median','GameScore_last64_max','GameScore_last64_std'
                          ]
teamGame = teamGame.merge(playerTeamGame, on=['Date','Team'], how='left')

playerTeamGame = playerGame.loc[playerGame['Position']=='F'].groupby(['Date','Team']).agg({
    'GameScore' : ['mean','median','max','std'],
    'GameScore_last8' : ['mean','median','max','std'],
    'GameScore_last16' : ['mean','median','max','std'],
    'GameScore_last32' : ['mean','median','max','std'],
    'GameScore_last64' : ['mean','median','max','std']
}).reset_index()
playerTeamGame.columns = ['Date','Team','GameScoreF_mean','GameScoreF_median','GameScoreF_max','GameScoreF_std',
                          'GameScoreF_last8_mean','GameScoreF_last8_median','GameScoreF_last8_max','GameScoreF_last8_std',
                          'GameScoreF_last16_mean','GameScoreF_last16_median','GameScoreF_last16_max','GameScoreF_last16_std',
                          'GameScoreF_last32_mean','GameScoreF_last32_median','GameScoreF_last32_max','GameScoreF_last32_std',
                          'GameScoreF_last64_mean','GameScoreF_last64_median','GameScoreF_last64_max','GameScoreF_last64_std'
                          ]
teamGame = teamGame.merge(playerTeamGame, on=['Date','Team'], how='left')

playerTeamGame = playerGame.loc[playerGame['Position']=='D'].groupby(['Date','Team']).agg({
    'GameScore' : ['mean','median','max','std'],
    'GameScore_last8' : ['mean','median','max','std'],
    'GameScore_last16' : ['mean','median','max','std'],
    'GameScore_last32' : ['mean','median','max','std'],
    'GameScore_last64' : ['mean','median','max','std']
}).reset_index()
playerTeamGame.columns = ['Date','Team','GameScoreD_mean','GameScoreD_median','GameScoreD_max','GameScoreD_std',
                          'GameScoreD_last8_mean','GameScoreD_last8_median','GameScoreD_last8_max','GameScoreD_last8_std',
                          'GameScoreD_last16_mean','GameScoreD_last16_median','GameScoreD_last16_max','GameScoreD_last16_std',
                          'GameScoreD_last32_mean','GameScoreD_last32_median','GameScoreD_last32_max','GameScoreD_last32_std',
                          'GameScoreD_last64_mean','GameScoreD_last64_median','GameScoreD_last64_max','GameScoreD_last64_std'
                          ]
teamGame = teamGame.merge(playerTeamGame, on=['Date','Team'], how='left')

In [39]:
def _add_lag(df, cols, lag, groupCol):
    new_cols = [col+'_last'+str(lag) for col in cols]
    df[new_cols] = df.groupby(groupCol)[cols].fillna(method='ffill').transform(lambda x: x.rolling(window=lag).mean())
    df[new_cols] = df.groupby(groupCol)[new_cols].shift(1)

    return df

In [40]:
teamGame = teamGame.sort_values(by=['DateInt','Game_Id'])
# add some lag features (rolling averages of past games)
teamGame.loc[teamGame['Date']>=today.strftime('%Y-%m-%d'), 'Playoffs'] = 0
df = _add_lag(teamGame.loc[teamGame['Playoffs']==0].copy(),
                    ['Goals','Shots','ShotAttempts','xG','Goals_5v5','Shots_5v5','ShotAttempts_5v5','xG_5v5',
                     'GameScore_mean','GameScore_median','GameScore_max','GameScore_std'], 8, 'Team')
df = _add_lag(df, ['Goals','Shots','ShotAttempts','xG','Goals_5v5','Shots_5v5','ShotAttempts_5v5','xG_5v5',
                     'GameScore_mean','GameScore_median','GameScore_max','GameScore_std'], 16, 'Team')
df = _add_lag(df, ['Goals','Shots','ShotAttempts','xG','Goals_5v5','Shots_5v5','ShotAttempts_5v5','xG_5v5',
                     'GameScore_mean','GameScore_median','GameScore_max','GameScore_std'], 32, 'Team')
df = _add_lag(df, ['Goals','Shots','ShotAttempts','xG','Goals_5v5','Shots_5v5','ShotAttempts_5v5','xG_5v5',
                     'GameScore_mean','GameScore_median','GameScore_max','GameScore_std'], 64, 'Team')

In [41]:
# shift GameScore columns
shift_cols = ['GameScore_last8_mean','GameScore_last8_median','GameScore_last8_max','GameScore_last8_std',
          'GameScore_last16_mean','GameScore_last16_median','GameScore_last16_max','GameScore_last16_std',
          'GameScore_last32_mean','GameScore_last32_median','GameScore_last32_max','GameScore_last32_std',
          'GameScore_last64_mean','GameScore_last64_median','GameScore_last64_max','GameScore_last64_std',
          'GameScoreF_last8_mean','GameScoreF_last8_median','GameScoreF_last8_max','GameScoreF_last8_std',
          'GameScoreF_last16_mean','GameScoreF_last16_median','GameScoreF_last16_max','GameScoreF_last16_std',
          'GameScoreF_last32_mean','GameScoreF_last32_median','GameScoreF_last32_max','GameScoreF_last32_std',
          'GameScoreF_last64_mean','GameScoreF_last64_median','GameScoreF_last64_max','GameScoreF_last64_std',
          'GameScoreD_last8_mean','GameScoreD_last8_median','GameScoreD_last8_max','GameScoreD_last8_std',
          'GameScoreD_last16_mean','GameScoreD_last16_median','GameScoreD_last16_max','GameScoreD_last16_std',
          'GameScoreD_last32_mean','GameScoreD_last32_median','GameScoreD_last32_max','GameScoreD_last32_std',
          'GameScoreD_last64_mean','GameScoreD_last64_median','GameScoreD_last64_max','GameScoreD_last64_std'
]
df[shift_cols] = df.groupby('Team')[shift_cols].shift(1)

In [42]:
# do a self-join to get features for the opposing team
df['teamGameVal'] = df.groupby(['Game_Id','Season','Date'])['xG'].rank("dense").replace(2, -1)
df_opp = df.copy()
df_opp['teamGameVal'] = df_opp['teamGameVal'] * -1
df_opp = df_opp.drop(['Win','DateInt','Playoffs','Home','teamGameRankOverall','teamGameRank'],1)
df_opp.columns = ['Game_Id','Date','Opp','Season',
                        'GoalsAgainst','ShotsAgainst','ShotAttemptsAgainst','xGAgainst',
                        'GoalsAgainst_5v5','ShotsAgainst_5v5','ShotAttemptsAgainst_5v5','xGAgainst_5v5','PenaltiesAgainst',
                        'OppElo','OppGameScore_mean','OppGameScore_median','OppGameScore_max','OppGameScore_std',
                        'OppGameScore_last8_mean','OppGameScore_last8_median','OppGameScore_last8_max','OppGameScore_last8_std',
                        'OppGameScore_last16_mean','OppGameScore_last16_median','OppGameScore_last16_max','OppGameScore_last16_std',
                        'OppGameScore_last32_mean','OppGameScore_last32_median','OppGameScore_last32_max','OppGameScore_last32_std',
                        'OppGameScore_last64_mean','OppGameScore_last64_median','OppGameScore_last64_max','OppGameScore_last64_std',
                        'OppGameScoreF_mean','OppGameScoreF_median','OppGameScoreF_max','OppGameScoreF_std',
                        'OppGameScoreF_last8_mean','OppGameScoreF_last8_median','OppGameScoreF_last8_max','OppGameScoreF_last8_std',
                        'OppGameScoreF_last16_mean','OppGameScoreF_last16_median','OppGameScoreF_last16_max','OppGameScoreF_last16_std',
                        'OppGameScoreF_last32_mean','OppGameScoreF_last32_median','OppGameScoreF_last32_max','OppGameScoreF_last32_std',
                        'OppGameScoreF_last64_mean','OppGameScoreF_last64_median','OppGameScoreF_last64_max','OppGameScoreF_last64_std',
                        'OppGameScoreD_mean','OppGameScoreD_median','OppGameScoreD_max','OppGameScoreD_std',
                        'OppGameScoreD_last8_mean','OppGameScoreD_last8_median','OppGameScoreD_last8_max','OppGameScoreD_last8_std',
                        'OppGameScoreD_last16_mean','OppGameScoreD_last16_median','OppGameScoreD_last16_max','OppGameScoreD_last16_std',
                        'OppGameScoreD_last32_mean','OppGameScoreD_last32_median','OppGameScoreD_last32_max','OppGameScoreD_last32_std',
                        'OppGameScoreD_last64_mean','OppGameScoreD_last64_median','OppGameScoreD_last64_max','OppGameScoreD_last64_std',
                        'OppGoals_last8','OppShots_last8','OppShotAttempts_last8','OppxG_last8',
                        'OppGoals_5v5_last8','OppShots_5v5_last8','OppShotAttempts_5v5_last8','OppxG_5v5_last8',
                        'OppGoals_last16','OppShots_last16','OppShotAttempts_last16','OppxG_last16',
                        'OppGoals_5v5_last16','OppShots_5v5_last16','OppShotAttempts_5v5_last16','OppxG_5v5_last16',
                        'OppGoals_last32','OppShots_last32','OppShotAttempts_last32','OppxG_last32',
                        'OppGoals_5v5_last32','OppShots_5v5_last32','OppShotAttempts_5v5_last32','OppxG_5v5_last32',
                        'OppGoals_last64','OppShots_last64','OppShotAttempts_last64','OppxG_last64',
                        'OppGoals_5v5_last64','OppShots_5v5_last64','OppShotAttempts_5v5_last64','OppxG_5v5_last64',
                        'OppGameScore_mean_last8','OppGameScore_median_last8','OppGameScore_max_last8','OppGameScore_std_last8',
                        'OppGameScore_mean_last16','OppGameScore_median_last16','OppGameScore_max_last16','OppGameScore_std_last16',
                        'OppGameScore_mean_last32','OppGameScore_median_last32','OppGameScore_max_last32','OppGameScore_std_last32',
                        'OppGameScore_mean_last64','OppGameScore_median_last64','OppGameScore_max_last64','OppGameScore_std_last64',
                        'teamGameVal'
                        ]
df = df.merge(df_opp, on=['Game_Id','Date','Season','teamGameVal'])
df = df.drop('teamGameVal',1)
df = df.loc[df['Team']!=df['Opp']]
df['EloDiff'] = df['Elo'] - df['OppElo']
df['EloDiff_538adj'] = (df['Elo'] - df['OppElo'] + (df['Home']*50) + ((df['Home']-1)*50))*(df['Playoffs']*0.25+1)

In [43]:
# add lag features for allowed stats
df = _add_lag(df, ['GoalsAgainst','ShotsAgainst','ShotAttemptsAgainst','xGAgainst',
                               'GoalsAgainst_5v5','ShotsAgainst_5v5','ShotAttemptsAgainst_5v5','xGAgainst_5v5'], 8, 'Team')
df = _add_lag(df, ['GoalsAgainst','ShotsAgainst','ShotAttemptsAgainst','xGAgainst',
                               'GoalsAgainst_5v5','ShotsAgainst_5v5','ShotAttemptsAgainst_5v5','xGAgainst_5v5'], 16, 'Team')
df = _add_lag(df, ['GoalsAgainst','ShotsAgainst','ShotAttemptsAgainst','xGAgainst',
                               'GoalsAgainst_5v5','ShotsAgainst_5v5','ShotAttemptsAgainst_5v5','xGAgainst_5v5'], 32, 'Team')
df = _add_lag(df, ['GoalsAgainst','ShotsAgainst','ShotAttemptsAgainst','xGAgainst',
                               'GoalsAgainst_5v5','ShotsAgainst_5v5','ShotAttemptsAgainst_5v5','xGAgainst_5v5'], 64, 'Team')

In [44]:
df['LastGame'] = df.groupby('Team')['Date'].shift(1)
df['RestDays'] = (pd.to_datetime(df['Date']) - pd.to_datetime(df['LastGame']))/np.timedelta64(1,'D')

In [45]:
# add game lines
lines = pd.read_csv('lines.csv')
lines['Team'] = lines['Team'].replace({
    'Pittsburgh':'PIT','TampaBay':'TBL','SeattleKraken':'SEA','Vegas':'VGK','NYRangers':'NYR','Washington':'WSH','Montreal':'MTL','Toronto':'TOR',
    'Vancouver':'VAN','Edmonton':'EDM','Chicago':'CHI','Colorado':'COL','Winnipeg':'WPG','Anaheim':'ANA','Ottawa':'OTT','Buffalo':'BUF',
    'Florida':'FLA','NYIslanders':'NYI','Carolina':'CAR','Dallas':'DAL','Arizona':'ARI','Columbus':'CBJ','Detroit':'DET','Nashville':'NSH',
    'LosAngeles':'LAK','NewJersey':'NJD','Philadelphia':'PHI','Minnesota':'MIN','Boston':'BOS','St.Louis':'STL','Calgary':'CGY','SanJose':'SJS',
    'Tampa Bay':'TBL','NY Islanders':'NYI','St. Louis':'STL','NY Rangers':'NYR','San Jose':'SJS','New Jersey':'NJD','Seattle Kraken':'SEA',
    'Los Angeles':'LAK','Arizonas':'ARI','Tampa':'TBL','Phoenix':'ARI'
})
lines.to_csv('lines.csv', index=False)

#df = df.drop(['Open','Close'],1)
df = df.merge(lines[['Season','Date','Team','Open','Close']], how='left', on=['Season','Date','Team'])

df['OpenImpliedPct'] = 0.
df['Open'] = df['Open'].astype(float)
df.loc[df['Open']<0, 'OpenImpliedPct'] = df.loc[df['Open']<0, 'Open']/(df.loc[df['Open']<0, 'Open']-100)
df.loc[df['Open']>0, 'OpenImpliedPct'] = 100/(100+df.loc[df['Open']>0, 'Open'])

df['CloseImpliedPct'] = 0.
df['Close'] = df['Close'].astype(float)
df.loc[df['Close']<0, 'CloseImpliedPct'] = df.loc[df['Close']<0, 'Close']/(df.loc[df['Close']<0, 'Close']-100)
df.loc[df['Close']>0, 'CloseImpliedPct'] = 100/(100+df.loc[df['Close']>0, 'Close'])

V1.0
features = ['Home','EloDiff',
            'ShotAttempts_last8','OppShotAttempts_last8',
            'Shots_last16','ShotAttempts_last16','OppShotAttempts_5v5_last16','OppShotAttempts_last16','OppxG_last16',
            'ShotAttempts_5v5_last32','OppShotAttempts_last32','OppxG_last32','Shots_last32','OppShots_5v5_last32',
            'Goals_last64','xG_last64','Goals_5v5_last64','OppShotAttempts_last64','OppxG_last64',
            'GameScore_mean_last8','GameScore_max_last32','GameScore_max_last16','GameScore_median_last64',
            'GameScore_std_last16','GameScore_std_last32','GameScore_std_last64','GameScore_max_last64',
            'GameScore_median_last32'
            ]

V2.0
features = ['EloDiff','EloDiff_538adj',
            'Shots_5v5_last8','ShotAttempts_5v5_last8',
            'OppxG_last16','OppShotAttempts_last16',
            'OppShotAttempts_5v5_last32','OppShots_5v5_last32','OppGoals_last32','OppShotAttempts_last32',
            'ShotsAgainst_last8','ShotsAgainst_5v5_last8','ShotAttemptsAgainst_5v5_last8',
            'GameScore_median_last8','GameScore_max_last8'
            ]

V3.0
features = ['EloDiff','EloDiff_538adj',
            'Shots_5v5_last8','ShotAttempts_5v5_last8',
            'OppxG_last16','OppShotAttempts_last16',
            'OppShotAttempts_5v5_last32','OppShots_5v5_last32','OppGoals_last32','OppShotAttempts_last32',
            'ShotsAgainst_last8','ShotsAgainst_5v5_last8','ShotAttemptsAgainst_5v5_last8',
            'GameScore_median_last8','GameScore_max_last8','GameScore_last64_std','GameScoreF_last64_max','GameScoreF_last64_std',
            'GameScoreD_last64_mean','GameScoreD_last64_median','GameScoreD_last64_max','OppGameScoreD_last64_std',
            'OppGameScoreF_last64_median','OppGameScoreD_last32_median','OppGameScoreD_last32_max','GameScoreD_last32_std',
            'GameScoreD_last32_median','GameScore_last16_mean','GameScore_last16_median','GameScoreF_last16_median',
            'GameScoreF_last16_max','OppGameScoreD_last16_mean','OppGameScoreF_last16_mean','OppGameScore_last16_mean',
            'OppGameScore_last16_std','OppGameScore_last16_median','OppGameScoreF_last8_max','OppGameScoreF_last8_median',
            'OppGameScoreD_last8_mean','OppGameScoreD_last8_median','GameScoreD_last8_mean','GameScoreD_last8_median',
            'GameScoreD_last8_max','GameScoreF_last8_mean','GameScore_last8_median','GameScore_last8_max'
            ]

In [46]:
scaler_line = pickle.load(open('scaler_line.pkl', 'rb'))
features = ['Open', 'OpenImpliedPct']
X_df_line = df.loc[~df[features].isnull().max(1)]
#X_df = df.loc[df['Date']==today.strftime('%Y-%m-%d')]
#X_df = df.loc[df['Date']>="2021-11-01"]
#X_df_line = X_df_line.loc[X_df_line['Season']==2021]
X_df_line = X_df_line.loc[X_df_line['Home']==1]
X = X_df_line[features].values
X = scaler_line.transform(X)

Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [47]:
model_line = pickle.load(open('lr_line.pkl', 'rb'))
X_df_line['winPct_line'] = model_line.predict_proba(X)[:,1]
X_df_line['OppWinPct_line'] = 1 - X_df_line['winPct_line']

Trying to unpickle estimator LogisticRegression from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [48]:
scaler = pickle.load(open('games_scaler.pkl', 'rb'))
features = ['Open', 'OpenImpliedPct', 'ShotAttempts_5v5_last8', 'OppShotAttempts_5v5_last16',
 'GameScoreF_last32_median', 'ShotsAgainst_5v5_last8', 'GoalsAgainst_5v5_last8', 'OppShotAttempts_last32',
 'ShotsAgainst_last8', 'EloDiff', 'OppShotAttempts_5v5_last8', 'ShotAttempts_last8', 'GameScore_last8_max',
 'GameScoreF_last8_max', 'GameScoreD_last8_max', 'RestDays', 'GameScore_last64_median', 'GameScore_last64_std',
 'GameScoreF_last64_std', 'GameScoreF_last16_mean', 'OppShots_5v5_last8', 'OppShots_last32',
 'EloDiff_538adj', 'CloseImpliedPct']
X_df = df.loc[~df[features].isnull().max(1)]
#X_df = df.loc[df['Date']==today.strftime('%Y-%m-%d')]
#X_df = df.loc[df['Date']>="2021-11-01"]
X_df = X_df.loc[X_df['Season']>=2020]
X_df = X_df.loc[X_df['Home']==1]
X = X_df[features].values
X = scaler.transform(X)

Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [49]:
model = pickle.load(open('games_lr.pkl', 'rb'))
X_df['winPct'] = model.predict_proba(X)[:,1]
X_df['OppWinPct'] = 1 - X_df['winPct']

Trying to unpickle estimator LogisticRegression from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [50]:
X_df_line = X_df_line[['Date','Team','winPct_line','OppWinPct_line']]
X_df = X_df.merge(X_df_line, how='outer', on=['Date','Team'])

In [51]:
X_df['adj_winPct'] = ((X_df['winPct'] - 0.5) * 1.1) + 0.5
X_df['adj_OppWinPct'] = 1 - X_df['adj_winPct']

#X_df['adj_winPct_line'] = ((X_df['winPct_line'] - 0.5) * 1.3) + 0.5
#X_df['adj_OppWinPct_line'] = 1 - X_df['adj_winPct_line']

In [52]:
X_df.loc[(X_df['Date']=="2022-01-19")][['Date','Team','Opp','Home','Win','Open','Close','winPct','adj_winPct']]

,Date,Team,Opp,Home,Win,Open,Close,winPct,adj_winPct
1456,2022-01-19,NJD,ARI,1.0,NaN,-200.0,-210.0,0.641288,0.655417
1457,2022-01-19,NYR,TOR,1.0,NaN,125.0,140.0,0.393569,0.382926
1458,2022-01-19,ANA,COL,1.0,NaN,215.0,210.0,0.293199,0.272519


In [51]:
from sklearn.metrics import log_loss, roc_auc_score
print ('LogLoss score: {}'.format(str(log_loss(X_df.loc[~X_df[['Win','winPct']].isnull().max(1)]['Win'].values,
                                               X_df.loc[~X_df[['Win','winPct']].isnull().max(1)]['winPct'].values))))
print ('AUC score: {}'.format(str(roc_auc_score(X_df.loc[~X_df[['Win','winPct']].isnull().max(1)]['Win'].values,
                                                X_df.loc[~X_df[['Win','winPct']].isnull().max(1)]['winPct'].values))))

LogLoss score: 0.6528006879581068
AUC score: 0.6559197695035461


In [52]:
X_df['adj_winPct_line'] = ((X_df['winPct_line'] - 0.5) * 1.1) + 0.5
X_df['adj_OppWinPct_line'] = 1 - X_df['adj_winPct_line']
print ('LogLoss score: {}'.format(str(log_loss(X_df.loc[(~X_df[['Win','adj_winPct_line']].isnull().max(1))&(X_df['Season']==2020)]['Win'].values,
                                               X_df.loc[(~X_df[['Win','adj_winPct_line']].isnull().max(1))&(X_df['Season']==2020)]['adj_winPct_line'].values))))
print ('AUC score: {}'.format(str(roc_auc_score(X_df.loc[(~X_df[['Win','adj_winPct_line']].isnull().max(1))&(X_df['Season']==2020)]['Win'].values,
                                                X_df.loc[(~X_df[['Win','adj_winPct_line']].isnull().max(1))&(X_df['Season']==2020)]['adj_winPct_line'].values))))
#0.6547814711136989

LogLoss score: 0.6564032582004178
AUC score: 0.6562658507894368


In [ ]:
X_df['adj_winPct'] = ((X_df['winPct'] - 0.5) * 1.2) + 0.5
X_df['adj_OppWinPct'] = 1 - X_df['adj_winPct']
print ('LogLoss score: {}'.format(str(log_loss(X_df.loc[~X_df[['Win','adj_winPct']].isnull().max(1)]['Win'].values,
                                               X_df.loc[~X_df[['Win','adj_winPct']].isnull().max(1)]['adj_winPct'].values))))
print ('AUC score: {}'.format(str(roc_auc_score(X_df.loc[~X_df[['Win','adj_winPct']].isnull().max(1)]['Win'].values,
                                                X_df.loc[~X_df[['Win','adj_winPct']].isnull().max(1)]['adj_winPct'].values))))
#0.6547814711136989

LogLoss score: 0.6513581075822018
AUC score: 0.6559197695035461


In [ ]:
X_df['binned_winPct'] = np.round(X_df['winPct'], 1)
#X_df.loc[X_df['Season']<=2021][['binned_winPct','winPct','Win']]\
#    .groupby(['binned_winPct']).agg({'winPct':['mean'],'Win':['count','mean']})
X_df.loc[X_df['Season']<=2021][['binned_winPct','winPct','adj_winPct','Win']]\
    .groupby(['binned_winPct']).agg({'winPct':['mean'],'adj_winPct':['mean'],'Win':['count','mean']})

winPct adj_winPct   Win          
                   mean       mean count      mean
binned_winPct                                     
0.2            0.241293   0.189551     2  0.000000
0.3            0.315602   0.278723    81  0.345679
0.4            0.407142   0.388571   280  0.410714
0.5            0.499183   0.499019   420  0.516667
0.6            0.598039   0.617646   345  0.640580
0.7            0.691363   0.729636   200  0.750000
0.8            0.761291   0.813549    24  0.875000

In [ ]:
lines = pd.read_csv('2021_lines.csv')

In [ ]:
lines.columns

Index(['date', 'fav', 'underdog', 'line_fav', 'line_underdog', 'winner'], dtype='object')

In [ ]:
faves = lines[['date','fav','line_fav','winner']]
faves['win'] = faves['fav']==faves['winner']
faves = faves.drop('winner',1)
faves.columns = ['Date','Team','Line','Win']

dogs = lines[['date','underdog','line_underdog','winner']]
dogs['win'] = dogs['underdog']==dogs['winner']
dogs = dogs.drop('winner',1)
dogs.columns = ['Date','Team','Line','Win']

lines = pd.concat([faves, dogs], ignore_index=True)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
lines['ImpliedPct'] = 0.
lines.loc[lines['Line']<0, 'ImpliedPct'] = lines.loc[lines['Line']<0, 'Line']/(lines.loc[lines['Line']<0, 'Line']-100)
lines.loc[lines['Line']>0, 'ImpliedPct'] = 100/(100+lines.loc[lines['Line']>0, 'Line'])

In [ ]:
def _calc_rel_win(lines, X_df, pct_col, pct_col_opp):
    #combine game lines with predictions
    home = X_df[['Date','Team',pct_col]]
    away = X_df[['Date','Opp',pct_col_opp]]
    away.columns = ['Date','Team',pct_col]
    tmpdf = pd.concat([home, away], ignore_index=True)
    tmpdf = lines.merge(tmpdf, how='inner', on=['Date','Team'])

    #find all games/teams that would have been bet on
    tmpdf = tmpdf.loc[tmpdf[pct_col]>tmpdf['ImpliedPct'], :]

    #calculate ratio of win to bet, assuming a win
    tmpdf['GainRatio'] = 0.
    tmpdf.loc[tmpdf['Line']<0, 'GainRatio'] = -100/tmpdf.loc[tmpdf['Line']<0, 'Line']
    tmpdf.loc[tmpdf['Line']>0, 'GainRatio'] = tmpdf.loc[tmpdf['Line']>0, 'Line']/100.

    #calculate bet amount
    tmpdf['NormBet'] = tmpdf[pct_col]-((1-tmpdf[pct_col])/tmpdf['GainRatio'])

    #try capping bets
    #tmpdf.loc[tmpdf['NormBet']>0.1, 'NormBet'] = 0.1

    #calculate net win
    tmpdf['NetWin'] = 0.
    tmpdf.loc[tmpdf['Win'], 'NetWin'] = tmpdf.loc[tmpdf['Win'], 'NormBet']*tmpdf.loc[tmpdf['Win'], 'GainRatio']
    tmpdf.loc[~tmpdf['Win'], 'NetWin'] = -tmpdf.loc[~tmpdf['Win'], 'NormBet']

    return tmpdf

In [ ]:
df_netWin = _calc_rel_win(lines, X_df, 'adj_winPct','adj_OppWinPct')
print (df_netWin['NetWin'].sum())
#3.5077276967942286 for 1.5 adj, 1.4412049759626608 for 0

1.233699583180425


In [ ]:
X_df['adj_winPct_line'] = ((X_df['winPct_line'] - 0.5) * 1.5) + 0.5
X_df['adj_OppWinPct_line'] = 1 - X_df['adj_winPct_line']

In [ ]:
X_df['adj_winPct_line'].describe()

count    10594.000000
mean         0.565031
std          0.139957
min         -0.145493
25%          0.474877
50%          0.575494
75%          0.662908
max          0.972433
Name: adj_winPct_line, dtype: float64

In [ ]:
df_netWin[['Date','NetWin']].groupby('Date').sum().reset_index().sort_values(by='Date')

,Date,NetWin
0,2021-10-12,0.449802
1,2021-10-13,0.182309
2,2021-10-14,-0.021058
3,2021-10-15,-0.012736
4,2021-10-16,0.138802
...,...,...
57,2021-12-15,-0.024080
58,2021-12-16,0.068854
59,2021-12-17,0.121541
60,2021-12-18,0.042535


In [ ]:
df_netWin.loc[df_netWin['Date']=="2021-12-19"]

,Date,Team,Line,Win,ImpliedPct,winPct_line,GainRatio,NormBet,NetWin
879,2021-12-19,STL,120,False,0.454545,0.460526,1.20,0.010965,-0.010965
880,2021-12-19,LAK,205,True,0.327869,0.429490,2.05,0.151192,0.309943
881,2021-12-19,NJD,180,False,0.357143,0.376692,1.80,0.030411,-0.030411


In [ ]:
df_netWin[['Team','NetWin']].groupby('Team').sum().sort_values(by='NetWin')

,NetWin
Team,
BOS,-0.308696
NYI,-0.274568
ARI,-0.267289
FLA,-0.255144
MTL,-0.162210
TBL,-0.112583
COL,-0.112260
VAN,-0.107854
WPG,-0.105197
